In [ ]:
# Front-end classification, bar plot, and umap
    # Post data structure, prior to split for mbatch and vae

#### Imports

In [1]:
import pandas as pd
import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import statistics

#### Classifier

In [ ]:
# glob ../output/dbl-cncr/

In [3]:
obj_n = pd.read_csv(
    '../output/dbl-cncr/pancreatic-adeno+lung-adeno_transcriptomics_cell-line+CPTAC.tsv',
    sep = '\t', index_col = 0)

In [ ]:
obj_n

In [5]:
X = obj_n.iloc[:, 2:]
y_sys = obj_n.System
y_c_typ = obj_n.Cancer_type

sys_scores = []
c_typ_scores = []

for i in list(range(0,9)):

    # System label predictions                             y_sys:
    X_train, X_test, y_train, y_test = train_test_split(X, y_sys)
    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = f1_score(y_test, y_pred, average = 'weighted')
    sys_scores.append(score)


    # Cross val index matching           y_c_typ:
    y_train = y_c_typ[y_c_typ.index.isin(y_train.index)]
    y_test = y_c_typ[y_c_typ.index.isin(y_test.index)]

    # Cancer label predictions
    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = f1_score(y_test, y_pred, average = 'weighted')
    c_typ_scores.append(score)

#### Bar plot of classification results

In [8]:
file_n = 'pancreatic-adeno+lung-adeno_transcriptomics_cell-line+CPTAC'

In [ ]:
sys_mean = statistics.mean(sys_scores)
c_typ_mean = statistics.mean(c_typ_scores)

sys_err = statistics.stdev(sys_scores)
c_typ_err = statistics.stdev(c_typ_scores)

fig, ax = plt.subplots()

# Create bars
bar1 = ax.bar('sys', sys_mean, yerr=sys_err, capsize = 7, label='System')
bar2 = ax.bar('c typ', c_typ_mean, yerr=c_typ_err, capsize = 7, label='Cancer type')

plt.title(file_n) # automate later

# Legend
ax.legend()
plt.savefig('../clf_plots/'+file_n+'rf.png')
# plt.show()

#### UMAP